In [ ]:
import pandas as pd
import altair as alt
import numpy as np

In [ ]:
def extrae_drive(url):
    edit = '/edit#gid='
    export ='/export?format=xlsx&gid='
    url = (url).replace(edit, export)
    return url

## Generando la Base principal
* Se usara la Planilla y la base de cesados, esta sera filtrada segun las condiciones de la conclusion
* En esta nueva base se empezara a agregar el resto de informacion para el proyecto

In [ ]:
cesados = extrae_drive('https://docs.google.com/spreadsheets/d/1bO_T0lAQgjcpjFFNVAwiXd2obwRjJS3K/edit#gid=2050623625')
cesados = pd.read_excel(
    cesados,
    usecols='E,I,O,P,S,T,W,X',
    dtype = {'CODIGO':str}
).rename(
    columns = {
        'Total o Temprana':'TIEMPO_CESE',
        'Producto':'PRODUCTO',
        'TIPO':'TIPO_CESE',
        'MOTIVO PRIMARIO':'MOTIVO_CESE'
    }
)

planilla = extrae_drive('https://docs.google.com/spreadsheets/d/1L094YBwj3BvobbXb4t3ahzpmL6gBSxgs/edit#gid=648976010')
planilla = pd.read_excel(
    planilla,
    usecols='A,B,D:H',
    dtype={'COD':str}
).rename(
    columns={
        'COD':'CODIGO',
        'REGION/DPTO':'REGION',
        'DIVISION/GC':'DIVISION'

    }
)

* Al aprecer hay colaboradores que cesaron mas de dos veces, es decir que ingresaron, cesaron, volvieron a ingresar y volvieron a cesar
* Se tomara en cuenta este numero de ceses como una caracteristca (aunque son pocas las personas)
* Eliminare los colaboradores duplicados considerando solo los ultimos ceses, pues son de los que se tendria data mas reciente
* Genero la columna antiguedad definida como CESE menos INGRESO en meses

In [ ]:
cesados.groupby(by = 'CODIGO', as_index=False).agg(n = ('CODIGO','count')).query('n>=2')

In [ ]:
cesados = cesados.sort_values(
    by = 'CESE',
    ascending = False # de mayor a menor
).drop_duplicates(
    'CODIGO',
    keep = 'first' # los mas recientes
).iloc[:, [0,2,3,6,7]]

In [ ]:
cesados['antiguedad_meses'] = (((cesados['CESE'] - cesados['INGRESO']).dt.days)/30).round(0)

* Utilizo como base Planilla que tiene un registro de colabores cesados y activos
* Elimino los duplicados pues, la planilla cuenta con registro de colaboradores por meses, me quedo solo con los mas recientes
* Realizo los filtros para generar la base que quiero

In [ ]:
planilla = planilla.drop_duplicates(
    'CODIGO',
    keep = 'last').iloc[:, 1:6] # no me interesa la fecha, solo los valores unicos del codigo

In [ ]:
df = pd.merge(
    planilla,
    cesados,
    on = 'CODIGO'
).query('PRODUCTO == "GRUPAL"')

* Genero una base para Grupal y Otra para invidual porque seran dos diferentes proyectos con caracteristicas diferentes
* Empiezo a filtrar, Cargo, Tipo de cese, motivo y antiguedad

In [ ]:
df = df[
    (df['CARGO'].isin(['VENTA INDIVIDUAL', 'VENTA GRUPAL']))
    & (df['TIPO_CESE'] == 'VOLUNTARIA')
    & (df['MOTIVO_CESE'].isin(['Personal Eventual', 'No le Gusto el Trabajo']))
    & (df['antiguedad_meses'] >= 9)]



programa_mejora = extrae_drive('https://docs.google.com/spreadsheets/d/1CKFlLpx248yvZS0INAawAA5e8NC2qdvV/edit#gid=1510069928')
memos = extrae_drive('https://docs.google.com/spreadsheets/d/1Q2pvICQWLAh8Rvou9FvH085KULXioDVK/edit#gid=238204385')
vacaciones = extrae_drive('https://docs.google.com/spreadsheets/d/1riJOPN5KUwV1962wO6mjPL7PMgL2v3_s/edit#gid=800276956')
descanso_medico = extrae_drive('https://docs.google.com/spreadsheets/d/1H_oENDjS0FyDNVmSg1199PrE6s8q6z7-/edit#gid=1138295584')
bonos_venta_i = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
bonos_venta_g = extrae_drive('https://docs.google.com/spreadsheets/d/1Vjh5S86c1r60L-CnuQB_m6JZPy223Z4p/edit#gid=100516943834575165411')
desempeno = extrae_drive()
reconocimientos = extrae_drive()
hijxs = extrae_drive()
datos_personales = extrae_drive()
recategorizaciones = extrae_drive()
cobertura_ofi = extrae_drive()
antg_oficina = extrae_drive()
gptw_oficina = extrae_drive()